In [8]:
import os
from random import randrange
from glob import glob

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import CheXpertDataset
from model import MultiLabelClassification
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG
from libauc.models import DenseNet121
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

In [2]:
# Get Data
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '..', 'data')
CheXpert_train_hidden_features_all = np.load(os.path.join(DATA_DIR,'CheXpert_train_hidden_features_all.npy'))
CheXpert_train_labels_all = np.load(os.path.join(DATA_DIR,'CheXpert_train_labels_all.npy'))
CheXpert_valid_hidden_features_all = np.load(os.path.join(DATA_DIR,'CheXpert_valid_hidden_features_all.npy'))
CheXpert_valid_labels_all = np.load(os.path.join(DATA_DIR,'CheXpert_valid_labels_all.npy'))
extra_valid_age_sex_df = pd.read_csv(os.path.join(DATA_DIR,'extra_valid_age_sex.csv'))
extra_valid_hidden_features = np.load(os.path.join(DATA_DIR,'extra_valid_hidden_features.npy'))
extra_valid_labels = np.load(os.path.join(DATA_DIR,'extra_valid_labels.npy'))
extra_valid_images = glob(os.path.join(DATA_DIR, 'extraValid', '*'))

In [3]:
# Add age and sex to hidden feature vectors
extra_valid_age_sex_df = extra_valid_age_sex_df.replace(['F', 'M', 'O'], [0, 1, 2])
extra_valid_age_sex_df = extra_valid_age_sex_df.apply(pd.to_numeric)
extra_valid_age_sex_np = extra_valid_age_sex_df.to_numpy()
hidden_with_age_sex = np.concatenate((extra_valid_hidden_features, extra_valid_age_sex_np), axis=1)
#hidden_with_age_sex = hidden_with_age_sex.astype('float32')
hidden_with_age_sex = extra_valid_hidden_features.astype('float32')

In [4]:
def set_all_seeds(SEED):
    # REPRODUCIBILITY
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed = 144
set_all_seeds(seed)

In [ ]:
model = DenseNet121(pretrained=True, last_activation=None, activations='relu', num_classes=5)
layer = model._modules.get('classifier')

In [ ]:
layer.weight.data

In [ ]:
print(layer)

In [5]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)

Using cache found in /home/mhermon/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
print(model)

In [6]:
layer = model.features[-1]

In [31]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [64]:
def get_vector(image_name):
    # 1. Load the image with Pillow library
    img = Image.open(image_name)
    img = img.convert('RGB')
    t_img = to_tensor(scaler(img))
    t_img = t_img.type(torch.DoubleTensor)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(t_img).unsqueeze(0))
    t_img = t_img.float()
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(1, 1024, 7, 7)
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return my_embedding


In [14]:
img = Image.open(extra_valid_images[0])

In [15]:
t_img = to_tensor(scaler(img))

In [17]:
t_img.type(torch.DoubleTensor)

tensor([[[50004., 48327., 46237.,  ..., 37015., 46486., 46552.],
         [49978., 48509., 46988.,  ..., 17882., 35756., 43854.],
         [49184., 47688., 46158.,  ...,  3089., 14803., 30172.],
         ...,
         [64601., 64531., 63940.,  ..., 65517., 65519., 65519.],
         [64754., 64660., 63696.,  ..., 65525., 65526., 65526.],
         [59581., 58286., 58066.,  ..., 59865., 59873., 59873.]]],
       dtype=torch.float64)

In [65]:
emb = get_vector(extra_valid_images[0])

RuntimeError: The size of tensor a (1024) must match the size of tensor b (7) at non-singleton dimension 3

In [32]:
img = Image.open(extra_valid_images[0])

In [33]:
img = np.array(img)

In [34]:
img.shape

(320, 320)

In [36]:
new = np.stack((img, img, img))

In [43]:
img[1][0]

49970

In [50]:
new = np.resize(new, (320, 320, 3))

In [53]:
new_pil = Image.fromarray(new)

TypeError: Cannot handle this data type: (1, 1, 3), <i4

In [66]:
!pip install img2vec_pytorch

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for img2vec-pytorch: filename=img2vec_pytorch-0.2.5-py3-none-any.whl size=4345 sha256=e2622945c67a69565ef464890f871ba66e71fc6e36f128f3e9f5d22a79796f16
  Stored in directory: /home/mhermon/.cache/pip/wheels/6f/0c/72/da8110c546d100dc5261f18b9803b4f73fb76d39672c5bd145
Successfully built img2vec-pytorch
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [67]:
!python -m img2vec_pytorch.test_img_to_vec

/usr/bin/python: No module named img2vec_pytorch


In [71]:
from img2vec_pytorch import Img2Vec
from PIL import Image

# Initialize Img2Vec with GPU
img2vec = Img2Vec(cuda=True)

# Read in an image (rgb format)
img = Image.open(extra_valid_images[0])
np_im = np.array(img)

In [73]:
b = np.repeat(np_im[:, :, np.newaxis], 3, axis=2)

In [78]:
img = scaler(img)
np_im = np.array(img)

In [80]:
b = np.repeat(np_im[:, :, np.newaxis], 3, axis=2)

In [81]:
b.shape

(1, 224, 3, 224)